# Senkrechter Wurf im Newton'schen Gravitationspotential

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook soll beim senkrechten Wurf neben dem Luftwiderstand auch die Erdanziehung berücksichtigt werden. Dabei wird über die lineare Näherung hinausgegangen und das volle Newton'sche Gravitationspotential berücksichtigt. Die zugehörige Bewegungsgleichung in dimensionsloser Form ist durch

$$\ddot r = - \alpha \vert\dot{r}\vert \dot{r} - \frac{1}{2 r^2} $$

gegeben, wobei die Konstante $\alpha > 0$ ist.

Die Lösung der Bewegungsgleichung erfolgt mit der SciPy-Routine `integrate.solve_ivp`, wobei die Berechnung der Bewegung durch eine Abbruchbedingung beendet wird.

### Importanweisungen

In [ ]:
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementation der Bewegungsgleichung

Zur numerischen Behandlung wandeln wir die eingangs angegebene Differentialgleichung 2. Ordnung in ein System von zwei gekoppelten Differentialgleichungen 1. Ordnung

\begin{align}
\dot r &= v \\
\dot v &= -\alpha\vert v\vert v - \frac{1}{2 r^2}
\end{align}

um und definieren eine entsprechende Funktion.

In [ ]:
def dx_dt(t, xvec, alpha):
    r, v = xvec
    return v, -alpha*abs(v)*v - 1/(2*r**2)

### Abbruchbedingung

Die Berechnung der Flugbahn soll nur bis zum Erreichen der Erdoberfläche erfolgen, also wenn die Punktmasse von oben kommend den dimensionslosen Abstand $r=1$ vom Erdmittelpunkt erreicht. Die folgende Funktion implementiert diese Abbruchbedingung.

In [ ]:
def height_zero(t, xvec, alpha):
    return xvec[0]-1

height_zero.terminal = True
height_zero.direction = -1

### Lösung des Differentialgleichungssystems

In der folgenden Funktion wird die Lösung des Differentialgleichungssystems mit Hilfe von `integrate.solve_ivp` berechnet. Neben der Funktion `dx_dt`, die die Zeitableitung berechnet, dem Zeitintervall und den Anfangsbedingungen werden noch drei weitere Argumente benötigt:
- `dense_output` erlaubt es uns, erst später die Zeitpunkte festzulegen, zu denen wir die Lösung berechnet haben wollen, wenn das Argument auf `True` gesetzt wird.
- `events` wird auf die Funktion `height_zero` gesetzt, die die Abbruchbedingung festlegt.
- `args` übernimmt ein Tupel mit zusätzlichen, in den Differentialgleichung auftretenden Parametern, hier `alpha`.

Das Ergebnis der Integration befindet sich in dem `OdeResult`-Objekt `solution`, von dem wir hier die folgenden Attribute verwenden:
- `solution.t_events` enthält den Zeitpunkt, an dem die Lösung endet
- `solution.sol` erlaubt es, die Lösung zu vorgegebenen Zeitpunkten auszuwerten.

Hat ein vorzeitiger Abbruch stattgefunden, weil die Erdoberfläche innerhalb des vorgegebenen Zeitintervalls erreicht wurde, wird die Endzeit auf den entsprechenden Zeitpunkt gesetzt. Die Funktion `np.linspace` erzeugt dann eine äquidistante Liste von Zeitpunkten, zu denen die Werte von Ort und Geschwindigkeit bestimmt werden. Abschließend werden die zugehörigen Arrays entpackt und zusammen mit den Zeitpunkten zurückgegeben.

In [ ]:
def trajectory(t_end, n_out, alpha, r_0, v_0):
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), [r_0, v_0],
        dense_output=True, args=(alpha,),
        events=height_zero)
    if solution.t_events[0].size > 0:
        t_end = solution.t_events[0][0]
    t_values = np.linspace(0, t_end, n_out)
    r, v = solution.sol(t_values)
    return t_values, r, v

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern können die folgenden Variablen eingestellt werden:
- `t_end`: maximaler Zeitraum, über den die Lösung berechnet wird
- `alpha`: dimensionsloser Parameter für den Luftwiderstand
- `v_0`: Anfangsgeschwindigkeit

Innerhalb der Funktion `plot_result` wird festgelegt, dass für die Darstellung der Trajektorie 200 Punkte verwendet werden und die Bewegung von der Erdoberfläche aus erfolgen soll. Auch diese Parameter können bei Bedarf angepasst werden.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=10, min=0, max=200, step=1,
                   description=r"$t_\text{end}$"),
               "alpha":
               widgets.FloatSlider(
                   value=50, min=0, max=50, step=0.1,
                   description=r"$\alpha$"),
               "v_0":
               widgets.FloatSlider(
                   value=1, min=0.01, max=1, step=0.01,
                   description="$v(0)$")
               }

@interact(**widget_dict)
def plot_result(t_end, alpha, v_0):
    n_out = 200
    r_0 = 1
    t_values, r_values, v_values = trajectory(
        t_end, n_out, alpha, r_0, v_0)

    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(t_values, v_values)
    ax1.set_xlabel("$t$")
    ax1.set_ylabel("$v$")

    ax2.plot(t_values, r_values)
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("$r$")